In [1]:
from Environment.custom_env import CustomEnvironment
from pettingzoo.test import parallel_api_test
from pettingzoo.mpe import simple_adversary_v3
from Models.ActorCritic import ActorCritic
from Models.SoftActorCritic import SoftActorCritic
import json
import torch
import numpy as np
from tqdm import tqdm

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [3]:
# f = open('Models/model_config.json')
# model_variables = json.load(f)
# f.close

In [4]:
# ac_0 = ActorCritic(model_variables)
# ac_1 = ActorCritic(model_variables)
# ac_2 = ActorCritic(model_variables)
# ac_3 = ActorCritic(model_variables)
# ac_4 = ActorCritic(model_variables)

# #agent_id to object mapping
# objects = {
    
#     0 : ac_0,
#     1 : ac_1,
#     2 : ac_2,
#     3 : ac_3,
#     4 : ac_4
# }  

In [5]:
env = CustomEnvironment(env_config=env_variables)
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [6]:
# parallel_api_test(env, num_cycles=1000)

In [7]:
# observations, infos = env.reset()
# while env.agents:
#     # print(env.timestep)
#     # this is where you would insert your policy
#     actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    
#     observations, rewards, terminations, truncations, infos = env.step(actions)

# env.close()

In [8]:
env.agents

AttributeError: 'CustomEnvironment' object has no attribute 'agents'

In [ ]:
terminations

In [ ]:
if(terminations):
    print("america ya")

In [ ]:
truncations

In [ ]:
episodes = 100

for i in tqdm(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",

    while(env.agents): #until there are any surviving agents 

        actions = {}
        logs = {}

        for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
            if i in env.agents:
                act, log_prob = env.agents_objects[i].brain1.select_action(state = curr_obs[i])
            else:
                actions[i] = env.num_actions
            
        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        
        for i in range(env.n_agents):

            if i in env.agents:
                env.agents_objects[i].brain1.push_to_buffer(curr_obs[i],actions[i],rewards[i],next_obs[i],logs[i],terminations[i])
                env.agents_objects[i].brain1.update_weights() 
            
            if i in env.justdie:
                env.justdie[i].brain1.push_to_buffer(curr_obs[i],actions[i],rewards[i],next_obs[i],logs[i],terminations[i])
                env.justdie[i].brain1.update_weights() 
                del env.justdie[i]
                
        
        curr_obs = next_obs
